<img align="right" width="300" src="https://data-services.hosting.nyu.edu/assets/libraries_short_color.png" alt="NYU Libraries Logo">

# Guidance for working with L2 political data voter files

**Overview:** The L2 Political files come in two parts for each state: Demographic indicators and Voter History. Both are .tab files (tab delimited) and will need to be extracted. It is likely that you'll need to use a command line tool to decompress each archive. 

For larger states, you will want to redact parts of these files by only loading the minimum number of variables that you need into memory, merge them together, then sample them in order to perform analysis.

This notebook is designed to provide some guidance on working with the data and creating extracts in a memory-efficient manner. It uses Python Libraries that are included with the Anacondas distribution, with the exception of [Pyjanitor](https://pyjanitor.readthedocs.io/index.html). In order to install it, run

`conda install pyjanitor -c conda-forge`

The conda distribution here seems to work the best (as opposed to pip). See the documentation for each of these libraries (especially Pandas) for more information. You will need to restart your Kernel and notebook before proceeding if you haven't already installed Pyjanitor. 

Users are expected to have agreed to the Terms of Use and should create redacted versions of the data that strip out identifying information and load only variables that are important for analysis. For the latest documentation, visit the [L2 Political LibGuide](https://guides.nyu.edu/l2political)

## I. Loading Python modules and L2 Data

Begin by importing some basic libraries to work with the data 

In [88]:
import pandas as pd
import numpy as np
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#import janitor

**Loading .tab files**

First, we read in the .tab file of demographic variables, declare the tab as the separator, and set the encoding

There are 646 columns in the demographic file, and it's better to set the datetype as string (labeled "object" in Pandas) for each of them and only load certian columns to cut down on memory usage.

Declare the columns you want; see the codebook for the full list and don't take PID columns.


In [2]:
selected_variables = ['LALVOTERID',
                      'Residence_Addresses_Latitude',
                      'Residence_Addresses_Longitude',
                      'Voters_Gender',
                      'Voters_FIPS']

## Change the file path below accordingly to point toward the relevant file. Note that this data loading step below
## will take the longest out of any step for your computer to process

state_demographic = pd.read_csv('VM2--VA--2020-03-01-DEMOGRAPHIC.tab', 
                                sep='\t', dtype=str, encoding='unicode_escape',
                                usecols=selected_variables)

## Preview of the first 5 rows of the data subset. 

state_demographic.head(5)

,LALVOTERID,Residence_Addresses_Latitude,Residence_Addresses_Longitude,Voters_Gender,Voters_FIPS
0,LALVA167958824,37.901764,-75.505997,F,001
1,LALVA516951467,37.901764,-75.505997,M,001
2,LALVA462558999,37.901343,-75.566436,F,001
3,LALVA462552402,37.901343,-75.566436,M,001
4,LALVA4161517,37.807510,-75.607740,M,001


It's also not a bad idea to test the length of the entire file and verify it with the number of records on the documentation

In [3]:
len(state_demographic)

5264320

You can also run .info to get a breakdown, including how much memory the data frame is eating up.

This will give you context for how much your computer can handle.

In [4]:
state_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5264320 entries, 0 to 5264319
Data columns (total 5 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   LALVOTERID                     object
 1   Residence_Addresses_Latitude   object
 2   Residence_Addresses_Longitude  object
 3   Voters_Gender                  object
 4   Voters_FIPS                    object
dtypes: object(5)
memory usage: 200.8+ MB


Next, we will read into memory the voter history file that corresponds to the state.

This time, we make sure to include the *LALVOTERID* column, and we will add whether or not they voted in 2016.

In [5]:
needed_variables = ['LALVOTERID', 'General_2016_11_08']

state_voterhistory = pd.read_csv('VM2--VA--2020-03-01-VOTEHISTORY.tab',
                                 sep='\t', dtype=str, encoding='unicode_escape',
                                 usecols=needed_variables)
                                
state_voterhistory.head(5)

,LALVOTERID,General_2016_11_08
0,LALVA167958824,Y
1,LALVA516951467,NaN
2,LALVA462558999,Y
3,LALVA462552402,Y
4,LALVA4161517,Y


In [6]:
## Again, I will want to run an .info command to test the size and memory

state_voterhistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5264320 entries, 0 to 5264319
Data columns (total 2 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   LALVOTERID          object
 1   General_2016_11_08  object
dtypes: object(2)
memory usage: 80.3+ MB


## II. Join of Demographic and Voter History Tables

Now that I have established a two reduced tables with a connection, I can join them (using the merge method in Pandas) based on the *LALVOTERID* common key


In [7]:
merged_file = pd.merge(state_voterhistory, state_demographic,
                       how='left', left_on='LALVOTERID', right_on='LALVOTERID')

merged_file.head(5)

,LALVOTERID,General_2016_11_08,Residence_Addresses_Latitude,Residence_Addresses_Longitude,Voters_Gender,Voters_FIPS
0,LALVA167958824,Y,37.901764,-75.505997,F,001
1,LALVA516951467,NaN,37.901764,-75.505997,M,001
2,LALVA462558999,Y,37.901343,-75.566436,F,001
3,LALVA462552402,Y,37.901343,-75.566436,M,001
4,LALVA4161517,Y,37.807510,-75.607740,M,001


## III. Sampling the Records

From here, you can sample the records. The frac argument is in percentage, so here, it's a one percent sample note that the sample function also guesses the dtype (which is proccess intensive) so it's better to do the sample after the final merge

In [8]:
state_sample = merged_file.sample(frac=0.01)

state_sample.head(5)

,LALVOTERID,General_2016_11_08,Residence_Addresses_Latitude,Residence_Addresses_Longitude,Voters_Gender,Voters_FIPS
4506497,LALVA4381855,Y,36.861190,-76.266420,F,710
3220820,LALVA1954212,Y,37.228351,-79.976485,M,161
745576,LALVA738960,Y,37.509310,-77.649680,M,041
2152787,LALVA471745169,Y,36.985720,-76.612630,M,093
3196996,LALVA398865370,NaN,37.241560,-79.995220,M,161


Again, print out the info to see that its a valid sample size (actually one percent) and that it is using much less memory

You'll know that the sampling worked if the index numbers are nonsequential and random also, you can test the length and info of your new sample

In [9]:
state_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52643 entries, 4506497 to 2917779
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LALVOTERID                     52643 non-null  object
 1   General_2016_11_08             36531 non-null  object
 2   Residence_Addresses_Latitude   52639 non-null  object
 3   Residence_Addresses_Longitude  52639 non-null  object
 4   Voters_Gender                  52643 non-null  object
 5   Voters_FIPS                    52643 non-null  object
dtypes: object(6)
memory usage: 2.8+ MB


## IV. Further Deidentifying Geocoded Locations: Jittering

There are many other filtering and cleaning steps to be taken before writing out a new file

For example, the coordinates can be identifying, even if you strip out all of the other PID variables, because they are geocoded to the address. You will want to jitter them if you intend to aggregate or visualize data.

First, print out the data types to make sure what they are:

In [10]:
state_sample.dtypes

LALVOTERID                       object
General_2016_11_08               object
Residence_Addresses_Latitude     object
Residence_Addresses_Longitude    object
Voters_Gender                    object
Voters_FIPS                      object
dtype: object

In [11]:
## In order to jitter the coordinates, you need to change them to a float

state_sample['Residence_Addresses_Latitude'] = pd.to_numeric(state_sample['Residence_Addresses_Latitude']) 
state_sample['Residence_Addresses_Longitude'] = pd.to_numeric(state_sample['Residence_Addresses_Longitude']) 

state_sample.dtypes

LALVOTERID                        object
General_2016_11_08                object
Residence_Addresses_Latitude     float64
Residence_Addresses_Longitude    float64
Voters_Gender                     object
Voters_FIPS                       object
dtype: object

The module pyjanitor introduces a jitter method for dataframes that can be used to displace the exact coordinates of a location.

We'll start with jittering the latitude. This library inserts a Gaussian distribution ("noise") equal to the standard deviation of the scale. The exact amount of the scale in jitter may need to change but this is a good starting place. You may want to test the results in a mapping software like QGIS or ArcMap before proceeding too far

In [12]:
jittered_lat = state_sample.jitter(
    column_name='Residence_Addresses_Latitude',
    dest_column_name='lat_jitter',
    scale=0.05,
    clip=None,
    random_state=None,
)

jittered_lat.head(5)

,LALVOTERID,General_2016_11_08,Residence_Addresses_Latitude,Residence_Addresses_Longitude,Voters_Gender,Voters_FIPS,lat_jitter
4506497,LALVA4381855,Y,36.861190,-76.266420,F,710,36.810290
3220820,LALVA1954212,Y,37.228351,-79.976485,M,161,37.255338
745576,LALVA738960,Y,37.509310,-77.649680,M,041,37.505897
2152787,LALVA471745169,Y,36.985720,-76.612630,M,093,36.955577
3196996,LALVA398865370,NaN,37.241560,-79.995220,M,161,37.181218


In [13]:
## Now we jitter the longitude

jittered_long = state_sample.jitter(
    column_name='Residence_Addresses_Longitude',
    dest_column_name='long_jitter',
    scale=0.05,
    clip=None,
    random_state=None,
)

jittered_long.head(5)

,LALVOTERID,General_2016_11_08,Residence_Addresses_Latitude,Residence_Addresses_Longitude,Voters_Gender,Voters_FIPS,lat_jitter,long_jitter
4506497,LALVA4381855,Y,36.861190,-76.266420,F,710,36.810290,-76.255566
3220820,LALVA1954212,Y,37.228351,-79.976485,M,161,37.255338,-80.044706
745576,LALVA738960,Y,37.509310,-77.649680,M,041,37.505897,-77.576737
2152787,LALVA471745169,Y,36.985720,-76.612630,M,093,36.955577,-76.638565
3196996,LALVA398865370,NaN,37.241560,-79.995220,M,161,37.181218,-79.913420


To complete the jittering process, make sure you drop off the actual Residence_Addresses columns.

With this last dataframe, you should have a one percent sample that has jittered lat long coordinates, voter gender, whether or not they voted in 2016, and their unique ID:


In [14]:
stripped_sample = jittered_long.drop(columns=['Residence_Addresses_Latitude', 'Residence_Addresses_Longitude'])

stripped_sample.head(5)

,LALVOTERID,General_2016_11_08,Voters_Gender,Voters_FIPS,lat_jitter,long_jitter
4506497,LALVA4381855,Y,F,710,36.810290,-76.255566
3220820,LALVA1954212,Y,M,161,37.255338,-80.044706
745576,LALVA738960,Y,M,041,37.505897,-77.576737
2152787,LALVA471745169,Y,M,093,36.955577,-76.638565
3196996,LALVA398865370,NaN,M,161,37.181218,-79.913420


## V. Saving the Resulting Dataframe to a CSV

Finally, you may want to write out the sample to a CSV, leaving out the index that Pandas creates and stipulating the encoding into utf-8

In [ ]:
stripped_sample.to_csv(r'virginia-onepercent-sample.csv', index=False, encoding='utf-8')

In the course of your analysis, you may want to repeat this process for multiple states and/or develop an aggregate roster of voters culled from a subset. You are encouraged to visit the tools section on the [L2 Political LibGuide at NYU Libraries](https://guides.nyu.edu/l2political/tools).

In general, you should have a data management plan with these files that includes the following elements:

- Secure any original .tab files you've downloaded by storing them in a single directory location, and then deleting the original .tab files once you've generated extracts
- Manage your extracted samples in a secure environment

Simple scripts and suggestions are available on the L2 Political LibGuide. 


## Saahil Test
### Dropping Private/Redundant columns

In [89]:
state_demographic_ak = pd.read_csv(r'C:\Users\saahi\Desktop\VM2--AK--2020-03-18-DEMOGRAPHIC.tab', 
                                sep='\t', dtype=str, encoding='unicode_escape')

In [90]:
def drop_private(df):
    # list of all columns that contain private information
    private_cols = ['VoterTelephones_Landline7Digit'
                    ,'VoterTelephones_LandlineUnformatted'
                    ,'VoterTelephones_CellPhoneFormatted'
                    ,'VoterTelephones_CellPhoneUnformatted'
                    ,'Voters_FirstName'
                    ,'Voters_MiddleName'
                    ,'Voters_LastName'
                    ,'Voters_NameSuffix'
                    ,'Residence_Addresses_AddressLine'
                    ,'Residence_Addresses_ExtraAddressLine'
                    ,'Residence_Addresses_ZipPlus4'
                    ,'Residence_Addresses_HouseNumber'
                    ,'Residence_Addresses_PrefixDirection'
                    ,'Residence_Addresses_StreetName'
                    ,'Residence_Addresses_Designator'
                    ,'Residence_Addresses_SuffixDirection'
                    ,'Residence_Addresses_ApartmentNum'
                    ,'Residence_Families_FamilyID'
                    ,'Mailing_Addresses_AddressLine'
                    ,'Mailing_Addresses_ExtraAddressLine'
                    ,'Mailing_Addresses_HouseNumber'
                    ,'Mailing_Addresses_PrefixDirection'
                    ,'Mailing_Addresses_StreetName'
                    ,'Mailing_Addresses_Designator'
                    ,'Mailing_Addresses_SuffixDirection'
                    ,'Mailing_Addresses_ApartmentNum'
                    ,'Mailing_Families_FamilyID'
                    ,'Voters_BirthDate'
                    ,'Residence_Addresses_City'
                    ,'DateConfidence_Description'
                    ,'Precinct'
                    ,'VoterTelephones_LandlineAreaCode'
                    ,'Residence_Addresses_State' 
                    ,'Residence_Addresses_LatLongAccuracy']
    
    indices_to_drop = []
    # getting the indices for columns containing private information
    for i in range(df.shape[1]):
        for j in range(len(private_cols)):
            if private_cols[j] == df.columns[i]:
                indices_to_drop.append(i)
    
    # dropping the relevant columns            
    df.drop(df.columns[indices_to_drop], axis = 1, inplace = True)
    
    return df

In [91]:
data = drop_private(state_demographic_ak)
data.shape[1]

612

### Threshold

In [92]:
def data_threshold(df):
    # Dropping all columns that have 75% or more null values
    dt = int(0.75*df.shape[0])
    df.dropna(axis=1, thresh=dt, inplace=True)
    return df

In [93]:
data = data_threshold(data)
data.shape[1]

195

In [94]:
# print(data.columns.values)
test_data  = data.copy()

## Cleaning Data
#### Data-type for all columns is 'object'. The following method converts all possible columns to numeric

In [129]:
# Function converts the obvious numeric columns into int/float
def prelim_numeric_converter(df):
    df.loc[:, df.columns.str.startswith('Commercial')] = df.loc[:, df.columns.str.startswith('Commercial')].replace(["$"], "", regex=True)
    cols = df.columns
    for c in cols:
        try:
            df[c] = pd.to_numeric(df[c])
        except:
            pass
    return df

In [130]:
data = prelim_numeric_converter(data)

#### The following methods, while resource heavy, relpace '\%' with "" for columns on Election Data:

In [137]:
# converting election results to numeric (resource heavy)
def election_numeric_converter(df):
    col_len = df.shape[1]-1
    df.loc[:, df.columns.str.startswith('Election')] = df.loc[:, df.columns.str.startswith('Commercial')].replace(["$"], "", regex=True)
    for i in range(df.loc[:, df.columns.str.startswith('Election')].shape[1]):
        try:
            df.iloc[:, col_len-i] = pd.to_numeric(df.iloc[:, col_len-i])
        except:
            pass
    return df

In [138]:
# This one is less efficient but maybe we could use str.replace instead of regex?
def election_numeric_converter_2(df):
    col_len = df.shape[1]-1
    for i in range(df.loc[:, df.columns.str.startswith('Election')].shape[1]):
        df.iloc[:, col_len-i]=df.iloc[:, col_len-i].str.replace('%','')
    for i in range(df.loc[:, df.columns.str.startswith('Election')].shape[1]):
        try:
            df.iloc[:, col_len-i] = pd.to_numeric(df.iloc[:, col_len-i])
        except:
            pass  
    return df

In [133]:
data = election_numeric_converter(data)

#### The following methods, while resource heavy, relpace '\$' with "" for columns on Commercial Data:

In [128]:

data.loc[:, data.columns.str.startswith('Commercial')] = data.loc[:, data.columns.str.startswith('Commercial')].replace(["$"], "", regex=True)
    

In [127]:
# to re-initiate the smaller dataset
data = test_data.copy()

In [139]:
data.dtypes.value_counts()

float64    159
object      29
int64        7
dtype: int64

In [108]:
data['ElectionReturns_P18PrecinctTurnoutRepublicans']

0         26.0
1         26.0
2         26.0
3         26.0
4         26.0
          ... 
522488    26.0
522489    26.0
522490    26.0
522491    26.0
522492    26.0
Name: ElectionReturns_P18PrecinctTurnoutRepublicans, Length: 522493, dtype: float64

In [45]:
data.loc[:, data.columns.str.startswith('Commercial')]
    

,CommercialData_DwellingType,CommercialData_EstHomeValue,CommercialData_EstimatedHHIncome,CommercialData_EstimatedHHIncomeAmount,CommercialData_PropertyType
0,NaN,NaN,NaN,NaN,Unknown
1,NaN,NaN,NaN,NaN,Unknown
2,Multi-Family Dwelling,NaN,$1000-14999,$13000,Apartment
3,Multi-Family Dwelling,NaN,$1000-14999,$13000,Apartment
4,NaN,NaN,NaN,NaN,Unknown
...,...,...,...,...,...
522488,NaN,$325000,NaN,NaN,Unknown
522489,Single Family Dwelling Unit,NaN,$100000-124999,$105000,Residential
522490,NaN,NaN,NaN,NaN,Unknown
522491,NaN,NaN,NaN,NaN,Unknown


## Imputation using Machine Learning

### Converting Binary to 1 & 0 
#### Can't figure out what NaN values are 0 and what are empty so Im just gonna not impute values for them

In [142]:
# run after dropping personal data and threshold
def knn_imputation(df):
    binary = []
    # Getting the indices for columns that are binary
    for i in range(df.shape[1]):
        if (df.iloc[ : , i].isnull().values.any()):
            if (len(df.iloc[ : , i].value_counts()) == 1):
                binary.append(i)
    # Temporarily drop binary columns 
    temp_data_1 = data.columns[binary].copy()            
    data.drop(data.columns[binary], axis = 1, inplace = True)
    
    imputer = IterativeImputer()
    imputer.fit(data)
    Xtrans = imputer.transform(data)